In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端机器学习：MLOps阶段2：实验：开始使用BigQuery ML训练

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_bqml_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
    <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_bqml_training.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab上运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_bqml_training.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概览

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端的MLOps生产。本教程涵盖了阶段2：实验阶段：开始使用BigQuery ML进行训练。

### 目标

在本教程中，您将学习如何使用`Vertex AI`进行`BigQueryML`训练。

此教程使用以下谷歌云机器学习服务：

- `BigQueryML 训练`
- `Vertex AI 模型资源`
- `Vertex AI Vizier`

执行的步骤包括：

- 在您的项目中创建一个本地的 BigQuery 表
- 训练一个 BigQuery ML 模型
- 评估 BigQuery ML 模型
- 将 BigQuery ML 模型导出为云模型
- 将导出的模型上传为`Vertex AI 模型`资源
- 使用`Vertex AI Vizier`对 BigQuery ML 模型进行超参数调整
- 将一个 BigQuery ML 模型自动注册到`Vertex AI 模型注册表`中

数据集

本教程使用的数据集是来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)的企鹅数据集。这个版本的数据集用于根据可用特征（如喙长、鳍深等）预测企鹅的物种。

### 费用
本教程使用 Google Cloud 的计费组件：

- Vertex AI
- Cloud Storage
- BigQuery

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage 价格](https://cloud.google.com/storage/pricing) 和 [BigQuery 价格](https://cloud.google.com/bigquery/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成费用估算。

## 安装

安装以下软件包以执行这个笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install the packages
! pip3 install --upgrade pyarrow \
                        google-cloud-aiplatform \
                        google-cloud-bigquery \
                        google-cloud-bigquery-storage $USER_FLAG -q

### 重新启动内核

安装了额外的包后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### 设置您的谷歌云项目

**无论您使用什么笔记本环境，以下步骤都是必须的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用用于计算/存储成本。

1. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI，BigQuery，Compute Engine和Cloud Storage API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,bigquery,compute_component,storage_component)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter运行以`!`为前缀的行作为shell命令，并将以`$`为前缀的Python变量插值到这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改 `REGION` 变量，该变量用于本笔记本的其余操作。以下是支持 Vertex AI 的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能在 Vertex AI 中使用多区域存储桶进行训练。并非所有区域都支持所有 Vertex AI 服务。

了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您处于直播教程会话中，您可能正在使用共享的测试账户或项目。为了避免用户在创建的资源中发生名称冲突，您可以为每个实例会话创建一个时间戳，并将其附加到您在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经过身份验证。请跳过此步骤。

**如果您正在使用Colab**，请运行以下单元格，并按照提示进行通过oAuth身份验证。

**否则**，请按照以下步骤操作：

在Cloud控制台中，前往[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

1. **点击创建服务账户**。

2. 在**服务账户名称**字段中输入名称，然后点击**创建**。

3. 在**授予此服务账户对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex AI"，然后选择**Vertex AI管理员**。在过滤框中输入"存储对象管理员"，然后选择**存储对象管理员**。

4. 点击创建。一个包含您密匙的JSON文件将下载到您的本地环境。

5. 在下方单元格中将您的服务账户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，下面的步骤都是必需的。**

当您在 Python 中初始化 Vertex SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源在各个会话之间保留的地方。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中是全局唯一的，包括您组织外的那些项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有当您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容来验证对您的云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

#### 服务账户

您可以使用服务账户创建Vertex AI管道作业。如果您不想使用项目的Compute Engine服务账户，将 `SERVICE_ACCOUNT` 设置为另一个服务账户ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### 引入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

### 初始化Python的Vertex AI和BigQuery SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

创建 BigQuery 客户端。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

### 设置硬件加速器

您可以为预测设置硬件加速器。

将变量 `DEPLOY_GPU/DEPLOY_NGPU` 设为使用支持 GPU 的容器镜像以及分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一个具有 4 个 Nvidia Telsa K80 GPU 的容器镜像分配给每个 VM，您可以指定：

    (aiplatform.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定 `(None, None)` 来使用一个在 CPU 上运行的容器镜像。

在 [此处](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators) 了解更多关于您所在地区的硬件加速器支持。

In [ ]:
import os

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

### 设置预构建容器

设置用于预测的预构建Docker容器映像。

- 将变量`TF`设置为容器映像的TensorFlow版本。例如，`2-1`代表版本2.1，`1-15`代表版本1.15。以下列表显示了一些可用的预构建映像：


有关最新列表，请参见[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

### 设置机器类型

接下来，设置用于预测的机器类型。

- 设置变量`DEPLOY_COMPUTE`以配置用于预测的虚拟机的计算资源。
  - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
  - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## BigQuery ML 简介

BigQuery ML（BQML）提供了在 BigQuery 中使用 SQL 语法训练 ML 表格模型（如分类和回归）的功能。

了解更多关于[BigQuery ML 文档](https://cloud.google.com/bigquery-ml/docs)。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.ml_datasets.penguins"
BQ_TABLE = "bigquery-public-data.ml_datasets.penguins"

### 创建BQ数据集资源

首先，在您的项目中创建一个空的数据集资源。

In [ ]:
BQ_DATASET_NAME = "penguins"
DATASET_QUERY = f"""CREATE SCHEMA {BQ_DATASET_NAME}
"""

job = bqclient.query(DATASET_QUERY)

### 训练 BigQuery ML 模型

接下来，您将从公共数据集 penguins 创建和训练一个 BigQuery ML 表格分类模型，并使用 `CREATE MODEL` 语句将模型存储在您的项目中。 模型配置在 `OPTIONS` 语句中指定如下：

- `model_type`：要训练的表格模型的类型和架构，例如，DNN 分类。
- `labels`：作为标签的列。

了解有关 [CREATE MODEL 语句](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create) 的更多信息。

In [ ]:
MODEL_NAME = "penguins"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='DNN_CLASSIFIER',
    labels = ['species']
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(job.errors, job.state)

while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

tblname = job.ddl_target_table
tblname = "{}.{}".format(tblname.dataset_id, tblname.table_id)
print("{} created in {}".format(tblname, job.ended - job.started))

### 评估训练好的 BigQuery ML 模型

接下来，检索已训练的 BigQuery ML 模型的评估结果。

了解更多关于 [ML.EVALUATE 函数](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate)。

In [ ]:
EVAL_QUERY = f"""
SELECT *
FROM
  ML.EVALUATE(MODEL {BQ_DATASET_NAME}.{MODEL_NAME})
ORDER BY  roc_auc desc
LIMIT 1"""

job = bqclient.query(EVAL_QUERY)
results = job.result().to_dataframe()
print(results)

从BigQuery ML导出模型

您在BigQuery ML中训练的模型是一个TensorFlow模型。接下来，您将以TF.SavedModel格式导出TensorFlow模型工件。

In [ ]:
param = f"{PROJECT_ID}:{BQ_DATASET_NAME}.{MODEL_NAME} {BUCKET_URI}/{MODEL_NAME}"
! bq extract -m $param

MODEL_DIR = f"{BUCKET_URI}/{BQ_DATASET_NAME}"
! gsutil ls $MODEL_DIR

## 将BigQuery ML模型上传到Vertex AI模型资源

最后，现在您已经导出了BigQuery ML模型，您可以将模型工件上传到Vertex AI模型资源，就像上传自定义训练模型一样。

以下是将BigQuery ML模型类型映射到其相应导出模型格式的部分列表：

'LINEAR_REG'<br/>
'LOGISTIC_REG' --> TensorFlow SavedFormat

'AUTOML_CLASSIFIER'<br/>
'AUTOML_REGRESSOR' --> TensorFlow SavedFormat

'BOOSTED_TREE_CLASSIFIER'<br/>
'BOOSTED_TREE_REGRESSOR' --> XGBoost格式

'DNN_CLASSIFIER'<br/>
'DNN_REGRESSOR'<br/>
'DNN_LINEAR_COMBINED_CLASSIFIER'<br/>
'DNN_LINEAR_COMBINED_REGRESSOR' --> TensorFlow Estimator

In [ ]:
model = aiplatform.Model.upload(
    display_name="penguins_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=True,
)

部署模型

接下来，将你的模型部署到在线预测中。要部署模型，你需要调用`deploy`方法，并提供以下参数：

- `deployed_model_display_name`：部署模型的可读名称。
- `traffic_split`：指定在端点上流量分配给该模型的百分比，以字典形式提供一个或多个键值对。
如果只有一个模型，则指定为{ "0": 100 }，其中"0"指的是上传的这个模型，而100表示100%的流量。
如果端点上已有现有模型，需要对流量进行分割，那么可以使用model_id来指定，如{ "0": percent, model_id: percent, ... }，这里model_id是一个指向部署端点上现有模型的模型id。所有百分比必须加起来等于100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要连接到工作副本的加速器数量。
- `starting_replica_count`：最初预留的计算实例数量。
- `max_replica_count`：要扩展到的最大计算实例数量。在本教程中，只提供了一个实例。

In [ ]:
DEPLOYED_NAME = "penguins-" + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

取消部署模型

当您完成预测后，您可以从 `Endpoint` 资源中取消部署模型。这将取消所有计算资源，并停止为部署的模型计费。

In [ ]:
endpoint.undeploy_all()

#### 删除 `Vertex AI Model` 资源

方法 'delete()' 将删除模型。

In [ ]:
model.delete()

### 删除 `BigQuery ML` 模型

接下来，删除模型的 `BigQuery ML` 实例。

In [ ]:
MODEL_QUERY = f"""
DROP MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
"""

job = bqclient.query(MODEL_QUERY)

### 超参数调整和训练BigQuery ML模型

接下来，您将使用 `Vertex AI Vizier` 服务对BigQuery ML表格分类模型进行超参数调整。超参数设置在 `OPTIONS` 语句中指定如下：

- `HPARAM_TUNING_ALGORITHM`：选择下一个试验参数的算法。
- `num_trials`：试验次数。
- `max_parallel_trials`：并行进行的试验次数。

了解更多关于[CREATE MODEL语句的超参数调整](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-hyperparameter-tuning)。

In [ ]:
MODEL_NAME = "penguins"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='DNN_CLASSIFIER',
    labels = ['species'],
    num_trials=10,
    max_parallel_trials=2,
    HPARAM_TUNING_ALGORITHM = 'VIZIER_DEFAULT'
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(job.errors, job.state)

while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

tblname = job.ddl_target_table
tblname = "{}.{}".format(tblname.dataset_id, tblname.table_id)
print("{} created in {}".format(tblname, job.ended - job.started))

### 评估 BigQuery ML 训练模型

接下来，检索已训练的 BigQuery ML 模型的模型评估结果。

了解更多关于[ML.EVALUATE 函数的信息](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate)。

In [ ]:
EVAL_QUERY = f"""
SELECT *
FROM
  ML.EVALUATE(MODEL {BQ_DATASET_NAME}.{MODEL_NAME})
ORDER BY  roc_auc desc
LIMIT 1"""

job = bqclient.query(EVAL_QUERY)
results = job.result().to_dataframe()
print(results)

### 删除 `BigQuery ML` 模型

接下来，删除模型的 `BigQuery ML` 实例。

In [ ]:
MODEL_QUERY = f"""
DROP MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
"""

job = bqclient.query(MODEL_QUERY)

### 使用可解释性训练 BigQuery ML 模型

接下来，您训练相同的 BigQuery ML 模型，但这次您通过添加选项来启用模型预测的 Vertex AI 可解释性：

- `ENABLE_GLOBAL_EXPLAIN`

In [ ]:
MODEL_NAME = "penguins"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='DNN_CLASSIFIER',
    labels = ['species'],
    ENABLE_GLOBAL_EXPLAIN = True
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(job.errors, job.state)

while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

tblname = job.ddl_target_table
tblname = "{}.{}".format(tblname.dataset_id, tblname.table_id)
print("{} created in {}".format(tblname, job.ended - job.started))

删除`BigQuery ML`模型

接下来，删除模型的`BigQuery ML`实例。

In [ ]:
MODEL_QUERY = f"""
DROP MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
"""

job = bqclient.query(MODEL_QUERY)

模型注册中心

另外，您可以在训练模型时隐式上传您的BigQuery ML模型作为“Vertex AI模型”资源，并导出和导入模型工件。在此方法中，在训练模型时，您可以添加额外的选项，告诉BigQuery ML自动上传和注册经过训练的模型作为“模型”资源。

### 设置权限以自动注册模型

您需要为BigQuery ML设置一些额外的IAM权限，以便在训练后自动上传并注册模型。根据您的服务帐户，以下权限设置可能会失败。在这种情况下，我们建议在Cloud Shell中执行权限设置。

了解更多关于[设置模型注册中心的权限](https://cloud.google.com/bigquery-ml/docs/managing-models-vertex)

In [ ]:
! gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT --role=roles/aiplatform.admin --condition=None

### 训练和注册模型

接下来，通过添加以下参数选项，训练模型并自动将模型注册到`Vertex AI Model Registry`：

- `model_registry`：设置为“vertex_ai”以指示自动注册到`Vertex AI Model Registry`。
- `vertex_ai_model_id`：已注册模型的人类可读的显示名称。
- `vertex_ai_model_version_aliases`：模型的备用名称。

In [ ]:
MODEL_NAME = "penguins"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='DNN_CLASSIFIER',
    labels = ['species'],
    model_registry="vertex_ai",
    vertex_ai_model_id="bqml_model_{TIMESTAMP}", 
    vertex_ai_model_version_aliases=["1"]
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(job.errors, job.state)

while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

tblname = job.ddl_target_table
tblname = "{}.{}".format(tblname.dataset_id, tblname.table_id)
print("{} created in {}".format(tblname, job.ended - job.started))

### 在`Vertex Model Registry`中查找模型

最后，您可以使用`Vertex AI Model`的list()方法并带有筛选查询来找到自动注册的模型。

In [ ]:
models = aiplatform.Model.list(filter="display_name=bqml_model_" + TIMESTAMP)
model = models[0]

print(model.gca_resource)

In [ ]:
models = aiplatform.Model.list()
for model in models:
    if model.gca_resource.display_name.startswith("bqml"):
        print(model.gca_resource.display_name)

删除`BigQuery ML`模型

接下来，删除该模型的`BigQuery ML`实例。

In [ ]:
MODEL_QUERY = f"""
DROP MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
"""

job = bqclient.query(MODEL_QUERY)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud
项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

将 `delete_storage` 设置为 `True` 以删除此笔记本中使用的Cloud Storage存储桶。

In [ ]:
# Delete the endpoint using the Vertex endpoint object
endpoint.undeploy_all()
endpoint.delete()

# Delete the model using the Vertex model object
try:
    model.delete()
except Exception as e:
    print(e)

# Delete the created BigQuery dataset
! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME

delete_storage = False
if delete_storage or os.getenv("IS_TESTING"):
    # Delete the created GCS bucket
    ! gsutil rm -r $BUCKET_URI